# Эксперимент

## Описание
Опишите цель эксперимента здесь.

## Методология
Опишите подход и методологию.

## Результаты
Опишите результаты эксперимента.


In [57]:
# Импорты
import sys
from pathlib import Path

# Добавляем корень проекта в путь (для ноутбуков в experiments/)
project_root = Path().resolve().parent.parent
sys.path.append(str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Загрузка данных через DataManager
from src.data.data_manager import DataManager
from src.data.feature_builder import build_targets_multi_horizon

dm = DataManager()

In [59]:
df = dm.get_clean()

 Clean данные загружены из кеша: 35064 строк


In [61]:
df

,Usage_kWh
DateTime,
2017-01-01 00:00:00,570.685479
2017-01-01 01:00:00,604.642705
2017-01-01 02:00:00,518.732113
2017-01-01 03:00:00,608.188829
2017-01-01 04:00:00,714.140572
...,...
2020-12-31 19:00:00,1093.896979
2020-12-31 20:00:00,976.242006
2020-12-31 21:00:00,960.551132


In [63]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List

def build_features_recursive_step1(
    df: pd.DataFrame,
    target_col: str = "Usage_kWh",
    freq: str = "h",
    lags: Tuple[int, ...] = (1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168),
    add_month_mean: bool = True,
    add_year_mean: bool = True,
) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Строит X(t) и y(t+1) для стратегии recursive (one-step ahead).

    Что входит в X(t):
      1) Календарные признаки:
         - hour (0..23)
         - day_of_week (0..6)
         - is_weekend (0/1)
         - peak (0/1): 1 если час в [6..13], иначе 0
         - peak_month (0/1): 1 если месяц Jan–Mar или Sep–Dec, иначе 0
      2) "Средние значения по месяцу и году" БЕЗ УТЕЧКИ:
         - month_mean_hist: среднее внутри текущего (year, month) по данным ДО момента t
         - year_mean_hist: среднее внутри текущего year по данным ДО момента t
         Реализация: expanding mean с shift(1), чтобы текущее значение y(t) не попадало в фичи.
      3) Лаги:
         - lag_k = y(t-k), только прошлые значения.

    Возвращает:
      X: признаки, выровненные и без NaN
      y_next: таргет y(t+1), выровненный под X (без NaN)

    Важно:
      - df.index должен быть datetime-like (DateTime).
      - freq='h' предполагает часовые данные.
      - asfreq(freq) может создать NaN при пропусках; дальше они отфильтруются.
    """
    # 0) порядок + частота
    s = df[[target_col]].copy()
    s = s.sort_index().asfreq(freq)

    y = s[target_col].astype(np.float32)

    # 1) календарные фичи
    X = pd.DataFrame(index=s.index)
    idx = X.index

    X["hour"] = idx.hour.astype(np.int8)
    X["day_of_week"] = idx.dayofweek.astype(np.int8)
    X["is_weekend"] = (idx.dayofweek >= 5).astype(np.int8)

    # peak: 6..13 включительно
    X["peak"] = ((idx.hour >= 6) & (idx.hour <= 13)).astype(np.int8)

    # peak_month: Jan-Mar (1-3) OR Sep-Dec (9-12)
    X["peak_month"] = (idx.month.isin([1, 2, 3, 9, 10, 11, 12])).astype(np.int8)

    # 2) "средние по месяцу и году" без утечки
    # month_mean_hist: внутри (year, month) считаем expanding mean и сдвигаем на 1 назад
    if add_month_mean:
        ym = pd.DataFrame({"y": y, "year": idx.year, "month": idx.month}, index=idx)
        # expanding mean внутри группы, затем shift(1) чтобы исключить y(t)
        month_mean_hist = (
            ym.groupby(["year", "month"])["y"]
            .expanding()
            .mean()
            .reset_index(level=[0, 1], drop=True)
            .shift(1)
        )
        X["month_mean_hist"] = month_mean_hist.astype(np.float32)

    if add_year_mean:
        yy = pd.DataFrame({"y": y, "year": idx.year}, index=idx)
        year_mean_hist = (
            yy.groupby("year")["y"]
            .expanding()
            .mean()
            .reset_index(level=0, drop=True)
            .shift(1)
        )
        X["year_mean_hist"] = year_mean_hist.astype(np.float32)

    # 3) лаги (строго прошлое)
    for lag in lags:
        X[f"lag_{lag}"] = y.shift(lag)

    # 4) таргет one-step: y(t+1)
    y_next = y.shift(-1)

    # 5) фильтрация валидных строк: чтобы были все фичи + y_next
    valid = X.notna().all(axis=1) & y_next.notna()
    X = X.loc[valid]
    y_next = y_next.loc[valid].astype(np.float32)

    # небольшая оптимизация типов
    float_cols = X.select_dtypes(include=["float64"]).columns
    if len(float_cols) > 0:
        X[float_cols] = X[float_cols].astype(np.float32)

    return X, y_next

LGBMRegressor

In [65]:
import numpy as np
import pandas as pd
from typing import Tuple, Dict
from sklearn.metrics import mean_absolute_error, mean_squared_error
from lightgbm import LGBMRegressor

def smape(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-8) -> float:
    """
    sMAPE в процентах.
    """
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred) + eps)
    return float(100.0 * np.mean(2.0 * np.abs(y_pred - y_true) / denom))

def time_split_index(
    index: pd.DatetimeIndex,
    train_ratio: float = 0.7,
    val_ratio: float = 0.15
) -> Tuple[pd.DatetimeIndex, pd.DatetimeIndex]:
    """
    Делит временной индекс на train и val (test не используем).
    Возвращает индексы train и val.
    """
    n = len(index)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    train_idx = index[:train_end]
    val_idx = index[train_end:val_end]
    return train_idx, val_idx

def _predict_recursive_h_steps(
    model,
    full_df: pd.DataFrame,
    target_col: str,
    start_time: pd.Timestamp,
    h: int,
    lags: Tuple[int, ...],
    freq: str = "h",
    add_month_mean: bool = True,
    add_year_mean: bool = True,
) -> float:
    """
    Делает один рекурсивный прогноз на горизонт h из точки start_time:
      - на каждом шаге строит X(t) из уже известных/предсказанных y
      - предсказывает y(t+1)
      - двигается дальше

    Возвращает прогноз значения на момент start_time + h*freq.

    Важно:
      - Чтобы корректно собирать лаги, у нас должен быть доступ к истории до start_time.
      - full_df должен содержать исходный y для истории; для будущих шагов используем predictions.
    """
    # локальный буфер значений y (история + предсказания)
    # будем хранить Series, где индекс — datetime, значения — y (float)
    y_hist = full_df[target_col].astype(np.float32).copy()

    current_t = start_time

    # h шагов one-step
    for step in range(h):
        # для X(t) нам нужен набор лагов y(t - lag)
        # и календарные фичи для текущего t
        idx = current_t

        feat = {}

        # calendar
        feat["hour"] = np.int8(idx.hour)
        feat["day_of_week"] = np.int8(idx.dayofweek)
        feat["is_weekend"] = np.int8(1 if idx.dayofweek >= 5 else 0)
        feat["peak"] = np.int8(1 if (idx.hour >= 6 and idx.hour <= 13) else 0)
        feat["peak_month"] = np.int8(1 if idx.month in [1,2,3,9,10,11,12] else 0)

        # month/year historical means without leakage:
        # считаем по истории ДО current_t (то есть по y_hist[:current_t - freq])
        # реализация простая (не самая быстрая, но прозрачная для эксперимента)
        hist_cutoff = current_t - pd.to_timedelta(1, unit=freq)
        y_past = y_hist.loc[:hist_cutoff].dropna()

        if add_month_mean:
            same_month = y_past[(y_past.index.year == idx.year) & (y_past.index.month == idx.month)]
            feat["month_mean_hist"] = np.float32(same_month.mean()) if len(same_month) else np.float32(np.nan)

        if add_year_mean:
            same_year = y_past[(y_past.index.year == idx.year)]
            feat["year_mean_hist"] = np.float32(same_year.mean()) if len(same_year) else np.float32(np.nan)

        # lags
        for lag in lags:
            lag_time = current_t - pd.to_timedelta(lag, unit=freq)
            feat[f"lag_{lag}"] = np.float32(y_hist.loc[lag_time]) if lag_time in y_hist.index else np.float32(np.nan)

        # если какие-то фичи не собраны (NaN) — прогноз невозможен
        # (обычно это случается в начале ряда, если не хватает истории)
        if any(pd.isna(v) for v in feat.values()):
            return np.nan

        X_t = pd.DataFrame([feat])
        y_next_pred = float(model.predict(X_t)[0])

        # записываем предсказание на t+1
        next_t = current_t + pd.to_timedelta(1, unit=freq)
        y_hist.loc[next_t] = np.float32(y_next_pred)

        current_t = next_t

    # после h шагов current_t == start_time + h
    return float(y_hist.loc[current_t])

def eval_recursive_one_step_strategy_val_only(
    df: pd.DataFrame,
    target_col: str = "Usage_kWh",
    horizons: Tuple[int, ...] = (1, 24, 168),
    freq: str = "h",
    lags: Tuple[int, ...] = (1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168),
    train_ratio: float = 0.7,
    val_ratio: float = 0.15,
    model_params: Dict = None,
    n_val_points: int = 500,  # чтобы не считать рекурсивно на тысячах точек (можешь увеличить)
    random_state: int = 42,
) -> pd.DataFrame:
    """
    Реализует стратегию "один шаг за раз" (recursive):
      - обучаем 1 модель на y(t+1)
      - на val делаем рекурсивные прогнозы для горизонтов h в horizons
      - считаем MAE / RMSE / sMAPE на val

    Особенности:
      - Рекурсивная оценка дорогая. Поэтому по умолчанию оцениваем на последних n_val_points точках val.
      - test не используем (как ты хотела для экспериментов).
    """
    if model_params is None:
        model_params = dict(
            n_estimators=800,
            learning_rate=0.05,
            num_leaves=63,
            max_depth=-1,
            subsample=0.8,
            colsample_bytree=0.8,
        )

    # 1) Собираем X и y_next (таргет t+1)
    X, y_next = build_features_recursive_step1(
        df=df,
        target_col=target_col,
        freq=freq,
        lags=lags,
        add_month_mean=True,
        add_year_mean=True,
    )

    # 2) time split по индексу X (и y_next уже выровнен под X)
    train_idx, val_idx = time_split_index(X.index, train_ratio=train_ratio, val_ratio=val_ratio)

    X_train = X.loc[train_idx]
    y_train = y_next.loc[train_idx]

    # val точки, из которых стартуем рекурсию:
    # Чтобы предсказать y(t+h), стартовая точка должна быть t, а y_true — на t+h.
    # Поэтому стартовые точки берём такие, чтобы t+h попадал в val-отрезок.
    full_series = df[[target_col]].copy().sort_index().asfreq(freq)
    val_start = val_idx.min()
    val_end = val_idx.max()

    results_rows = []

    # ограничим количество стартовых точек для скорости
    # берем последние n_val_points из val_idx (обычно логично)
    candidate_starts = val_idx
    if len(candidate_starts) > n_val_points:
        candidate_starts = candidate_starts[-n_val_points:]

    # 3) обучаем one-step модель
    model = LGBMRegressor(
        objective="regression",
        random_state=random_state,
        n_jobs=1,
        verbose=-1,
        **model_params
    )
    model.fit(X_train, y_train)

    # 4) оценка для каждого горизонта
    for h in horizons:
        y_true_list = []
        y_pred_list = []

        for t in candidate_starts:
            t_h = t + pd.to_timedelta(h, unit=freq)

            # true должен быть внутри val
            if t_h < val_start or t_h > val_end:
                continue

            # рекурсивный прогноз
            pred = _predict_recursive_h_steps(
                model=model,
                full_df=full_series,
                target_col=target_col,
                start_time=t,
                h=h,
                lags=lags,
                freq=freq,
                add_month_mean=True,
                add_year_mean=True,
            )

            if np.isnan(pred):
                continue

            true_val = float(full_series.loc[t_h, target_col])

            y_true_list.append(true_val)
            y_pred_list.append(pred)

        y_true_arr = np.array(y_true_list, dtype=float)
        y_pred_arr = np.array(y_pred_list, dtype=float)

        mae = mean_absolute_error(y_true_arr, y_pred_arr)
        rmse = np.sqrt(mean_squared_error(y_true_arr, y_pred_arr))
        smape_pct = smape(y_true_arr, y_pred_arr)

        results_rows.append({
            "strategy": "recursive_one_step",
            "horizon": h,
            "val_MAE": mae,
            "val_RMSE": rmse,
            "val_sMAPE_pct": smape_pct,
            "n_eval_points": len(y_true_arr),
            "n_features": X.shape[1],
            "lags": list(lags),
        })

        print(f"[Recursive one-step] h={h}: val_MAE={mae:.2f}, val_RMSE={rmse:.2f}, val_sMAPE={smape_pct:.2f}% | n={len(y_true_arr)}")

    return pd.DataFrame(results_rows).sort_values("horizon").reset_index(drop=True)

In [67]:
# df — твой clean_df с DateTime индексом и колонкой Usage_kWh

results_recursive = eval_recursive_one_step_strategy_val_only(
    df=df,
    target_col="Usage_kWh",
    horizons=(1, 24, 168),
    lags=(1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168),
    train_ratio=0.7,
    val_ratio=0.15,
    n_val_points=500  # можешь увеличить, если хватает времени
)

results_recursive

[Recursive one-step] h=1: val_MAE=103.23, val_RMSE=136.96, val_sMAPE=12.12% | n=499
[Recursive one-step] h=24: val_MAE=109.09, val_RMSE=144.74, val_sMAPE=12.82% | n=476
[Recursive one-step] h=168: val_MAE=116.16, val_RMSE=155.12, val_sMAPE=13.41% | n=332


,strategy,horizon,val_MAE,val_RMSE,val_sMAPE_pct,n_eval_points,n_features,lags
0,recursive_one_step,1,103.230513,136.955795,12.118952,499,18,"[1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168]"
1,recursive_one_step,24,109.093469,144.738843,12.819638,476,18,"[1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168]"
2,recursive_one_step,168,116.162363,155.121872,13.410627,332,18,"[1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168]"


In [69]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- если этих функций нет в текущем ноутбуке, убедись что они уже выполнены:
# smape(...)
# time_split_index(...)
# build_features_recursive_step1(...)
# _predict_recursive_h_steps(...)

def eval_recursive_one_step_strategy_val_only_rf(
    df: pd.DataFrame,
    target_col: str = "Usage_kWh",
    horizons: Tuple[int, ...] = (1, 24, 168),
    freq: str = "h",
    lags: Tuple[int, ...] = (1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168),
    train_ratio: float = 0.7,
    val_ratio: float = 0.15,
    rf_params: Dict = None,
    n_val_points: int = 500,     # ограничение точек валидации для скорости
    random_state: int = 42,
) -> pd.DataFrame:
    """
    RandomForest + recursive стратегия "один шаг за раз":

    1) Строим признаки X(t) и таргет y(t+1) (one-step) через build_features_recursive_step1.
    2) Делим по времени на train и val.
    3) Обучаем одну модель RandomForest на y(t+1).
    4) На val оцениваем горизонты:
         - h=1: один шаг
         - h=24: 24 последовательных one-step прогнозов
         - h=168: 168 последовательных one-step прогнозов
       и считаем MAE/RMSE/sMAPE.

    Важно:
    - Рекурсивная оценка вычислительно дорогая, поэтому n_val_points ограничивает число стартовых точек.
    - Test не используется (подходит для экспериментов с фичами).
    """
    if rf_params is None:
        rf_params = dict(
            n_estimators=400,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            max_features="sqrt",
            bootstrap=True,
            n_jobs=-1,               # распараллеливание внутри леса
        )

    # 1) X(t) и y(t+1)
    X, y_next = build_features_recursive_step1(
        df=df,
        target_col=target_col,
        freq=freq,
        lags=lags,
        add_month_mean=True,
        add_year_mean=True,
    )

    # 2) time split по индексу X
    train_idx, val_idx = time_split_index(X.index, train_ratio=train_ratio, val_ratio=val_ratio)

    X_train = X.loc[train_idx]
    y_train = y_next.loc[train_idx]

    # 3) для "истины" и для рекурсивной сборки лагов нужен полный ряд (частота + сортировка)
    full_series = df[[target_col]].copy().sort_index().asfreq(freq)

    val_start = val_idx.min()
    val_end = val_idx.max()

    # ограничиваем число стартовых точек (обычно берём последние)
    candidate_starts = val_idx
    if len(candidate_starts) > n_val_points:
        candidate_starts = candidate_starts[-n_val_points:]

    # 4) обучаем RF на one-step
    model = RandomForestRegressor(
        random_state=random_state,
        **rf_params
    )
    model.fit(X_train, y_train)

    # 5) оценка по горизонтам
    rows = []

    for h in horizons:
        y_true_list = []
        y_pred_list = []

        for t in candidate_starts:
            t_h = t + pd.to_timedelta(h, unit=freq)

            # true должна лежать внутри val
            if (t_h < val_start) or (t_h > val_end):
                continue

            pred = _predict_recursive_h_steps(
                model=model,
                full_df=full_series,
                target_col=target_col,
                start_time=t,
                h=h,
                lags=lags,
                freq=freq,
                add_month_mean=True,
                add_year_mean=True,
            )

            if np.isnan(pred):
                continue

            true_val = float(full_series.loc[t_h, target_col])

            y_true_list.append(true_val)
            y_pred_list.append(pred)

        y_true_arr = np.array(y_true_list, dtype=float)
        y_pred_arr = np.array(y_pred_list, dtype=float)

        mae = mean_absolute_error(y_true_arr, y_pred_arr)
        rmse = np.sqrt(mean_squared_error(y_true_arr, y_pred_arr))
        smape_pct = smape(y_true_arr, y_pred_arr)

        rows.append({
            "strategy": "recursive_one_step",
            "model": "RandomForest",
            "horizon": h,
            "val_MAE": mae,
            "val_RMSE": rmse,
            "val_sMAPE_pct": smape_pct,
            "n_eval_points": len(y_true_arr),
            "n_features": X.shape[1],
            "lags": list(lags),
            "rf_params": dict(rf_params),
        })

        print(
            f"[RF Recursive one-step] h={h}: "
            f"val_MAE={mae:.2f}, val_RMSE={rmse:.2f}, val_sMAPE={smape_pct:.2f}% | n={len(y_true_arr)}"
        )

    return pd.DataFrame(rows).sort_values("horizon").reset_index(drop=True)

In [71]:
results_rf_recursive = eval_recursive_one_step_strategy_val_only_rf(
    df=df,
    target_col="Usage_kWh",
    horizons=(1, 24, 168),
    lags=(1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168),
    train_ratio=0.7,
    val_ratio=0.15,
    n_val_points=500,
    rf_params=dict(
        n_estimators=500,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        bootstrap=True,
        n_jobs=-1
    )
)

results_rf_recursive

[RF Recursive one-step] h=1: val_MAE=102.62, val_RMSE=135.72, val_sMAPE=12.08% | n=499
[RF Recursive one-step] h=24: val_MAE=107.82, val_RMSE=143.43, val_sMAPE=12.65% | n=476
[RF Recursive one-step] h=168: val_MAE=114.64, val_RMSE=153.42, val_sMAPE=13.27% | n=332


,strategy,model,horizon,val_MAE,val_RMSE,val_sMAPE_pct,n_eval_points,n_features,lags,rf_params
0,recursive_one_step,RandomForest,1,102.616603,135.724665,12.084652,499,18,"[1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168]","{'n_estimators': 500, 'max_depth': None, 'min_..."
1,recursive_one_step,RandomForest,24,107.818210,143.425787,12.652967,476,18,"[1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168]","{'n_estimators': 500, 'max_depth': None, 'min_..."
2,recursive_one_step,RandomForest,168,114.638847,153.418761,13.268746,332,18,"[1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 168]","{'n_estimators': 500, 'max_depth': None, 'min_..."
